In [1]:
import cv2
import dlib
import matplotlib.pyplot as plt
import os
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
infile = open('/content/drive/MyDrive/x_file','rb')
X = pickle.load(infile)
infile.close()

# img_size1 = 128*4
# img_size2 = 64*4
img_size = 48

X = np.array(X).reshape(-1, img_size, img_size, 3)

infile = open('/content/drive/MyDrive/y_file','rb')
y = pickle.load(infile)
infile.close()

Y = np.array(y)

In [6]:
hog_features = []
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")

for img_array in X:
  try:
    resized_img = cv2.resize(img_array, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
    
    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) ==0:
      face = dlib.rectangle(0, 0, 128*4, 64*4)
      x1 = face.left()
      y1 = face.top()
      x2 = face.right()
      y2 = face.bottom()
      landmarks = predictor(image=gray, box=face)
      for n in range(0, 68):
          x = landmarks.part(n).x
          y = landmarks.part(n).y
            
      else:
          for face in faces:
              x1 = face.left() # left point
              y1 = face.top() # top point
              x2 = face.right() # right point
              y2 = face.bottom() # bottom point
              landmarks = predictor(image=gray, box=gray)

      for i in range(0, 68):
          fd = np.insert(fd,0, landmarks.part(i).x)
          fd = np.insert(fd,0, landmarks.part(i).y)


      hog_features.append(fd)
      if len(hog_features)== 3000:
        break

  except Exception as e:
        print(e)



In [7]:
len(hog_features)

3000

In [10]:
from sklearn.model_selection import train_test_split
clf = svm.SVC()
X = np.array(hog_features)

In [11]:
y=Y[0:3000]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [13]:
clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
y_pred = clf.predict(X_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.145


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.00      0.00      0.00        43
           2       0.00      0.00      0.00        88
           3       0.14      1.00      0.25        87
           4       0.00      0.00      0.00        88
           5       0.00      0.00      0.00        97
           6       0.00      0.00      0.00       110

    accuracy                           0.14       600
   macro avg       0.02      0.14      0.04       600
weighted avg       0.02      0.14      0.04       600



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
